In [114]:
import os, re, json
import cdsapi
import geopandas as gpd
from shapely.geometry import Polygon, mapping
from datetime import datetime
from timezonefinder import TimezoneFinder
import pytz
from zoneinfo import ZoneInfo
from zipfile import ZipFile

In [ ]:
# To get root path under project folder structure
cwd = os.path.dirname(os.path.abspath("__file__"))
root_folder = cwd.split('\\')
root_folder = root_folder[1:-3]
if root_folder[-1] != 'fireRunSeverity':
    print("!!-- Didn't get correct root folder! --!!")
    print(root_folder)
    print("!!-- Modify rooting index at line: 18 and come back --!!")
root_folder = os.path.join(os.sep,*root_folder)
run_DataDir = r"data/fireruns"
ER5_Dir     = r"data/ER5"
LandCV_Dir  = r"data/LandCover"

#load credential
cred_json = os.path.join(cwd, ".cds_cred.json")
if os.path.exists(cred_json):
    with open(cred_json, "r") as f1:
        cred = json.load(f1)

if "cred" in locals():
    cdsAPI_KEY = cred['cdsAPI_KEY']
else:
    cdsAPI_KEY = input("CDS API Key: ")
    cred = {"cdsAPI_KEY": cdsAPI_KEY}
    with open(cred_json, "w") as f1:
        json.dump(cred, f1)
del cred

# Local Side data
# List in put data for GEE fetch
AreaList = os.listdir(os.path.join(root_folder,run_DataDir))
AreaList[1]

## Load Local side data

In [ ]:
in_Name = AreaList[1]
print("\n\n ######################################################")
print("Process AOI:", in_Name)
dataFLD = os.path.join(root_folder, run_DataDir, in_Name)
inFLD = os.path.join(dataFLD, 'input')
shpIn  = []
for f in os.scandir(inFLD):
    if re.search(r".shp$", f.name):
        print("Shp Name:", f.name)
        shpIn.append(f)

if len(shpIn) != 1:
    print("No. of shp file not equal to 1!")
    exit()
else:
    shpIn = shpIn[0]

# Read Shp file
shpGPD = gpd.read_file(os.path.join(inFLD, shpIn))
print("Shp  CRS:", shpGPD.crs)
shpGPD_bf = shpGPD.buffer(10000)
shpbf_reproj = shpGPD_bf.to_crs("EPSG:4326")
del shpGPD_bf

# Get the bounds
minx, miny, maxx, maxy = shpbf_reproj.total_bounds
# Create a polygon of bounds
bounds_polygon = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])
bounds_wgs     = bounds_polygon.bounds
# Shapely bounds: [xmin, ymin, xmax, ymax]
# Reorder to CDS format [ymax, xmin, ymin, xmax]
print(bounds_polygon.bounds)
cds_bound = [bounds_wgs[3], bounds_wgs[0], bounds_wgs[1], bounds_wgs[2]]
print(cds_bound)



 ######################################################
Process AOI: isocronas_las_tablas
Shp Name: isocronas_las_tablas.shp
Shp  CRS: EPSG:32719
(-71.63790600801836, -33.291039620278845, -71.33041368117995, -32.892482205844416)
[-32.892482205844416, -71.63790600801836, -33.291039620278845, -71.33041368117995]


## Modify time zone

In [108]:
latitude = (miny + maxy)/2
longitude = (minx + maxx)/2
tf = TimezoneFinder()
timezone = tf.timezone_at(lng=longitude, lat=latitude)
print(f"Accurate Time Zone: {timezone}")

Accurate Time Zone: America/Santiago


In [18]:
print("Process AOI:", in_Name, end="\t")
dataFLD = os.path.join(root_folder, run_DataDir, in_Name)
inFLD = os.path.join(dataFLD, 'input')
shpIn  = []
for f in os.scandir(inFLD):
    if re.search(r".shp$", f.name):
        # print("Shp Name:", f.name)
        shpIn.append(f)

if len(shpIn) != 1:
    print("No. of shp file not equal to 1!")
    exit()
else:
    shpIn = shpIn[0]

shpGPD = gpd.read_file(os.path.join(inFLD, shpIn))

# Get date of data
# Convert time zone to UTC
fire_dtString = list(shpGPD['FeHo'].dropna())
fire_dtsList  = [datetime.strptime(dt, "%Y/%m/%d_%H%M").replace(tzinfo=ZoneInfo(timezone)) for dt in fire_dtString if not re.search("\\D", dt[:4])] # Filter out string with non-digit characters
tz_utc = pytz.timezone("UTC")
dateSt_UTC = min(fire_dtsList).astimezone(tz_utc)

# print(sorted(fire_dtsList_UTC))
dateSt = dateSt_UTC.date()
print(dateSt)

Process AOI: isocronas_las_tablas	2024-02-02


In [ ]:
# Check LandCover Folder
outFld = os.path.join(root_folder, LandCV_Dir, in_Name)
if not os.path.exists(outFld):
    os.makedirs(outFld)

c = cdsapi.Client(key=cdsAPI_KEY,
                    url='https://cds.climate.copernicus.eu/api', quiet=False, sleep_max=5, timeout=7200)

output_file = os.path.join(outFld, f"CDS_LandCover_{in_Name}_{dateSt.year}.nc")

if not os.path.exists(output_file):
  zip_name = output_file.replace(".nc", ".zip")
  cds_year = 2022 if dateSt.year > 2022 else dateSt.year
  dt_vers  = "v2_0_7cds" if cds_year < 2016 else "v2_1_1"
  print("Use Land Cover Map Year:", cds_year)
  
  if not os.path.exists(zip_name):
    print("-- Requesting Land Cover datasets...")
    dataset = "satellite-land-cover"
    request = {
      "variable": "all",
      "year": [str(cds_year)],
      "version": [dt_vers],
      "area": cds_bound,
    }

    c.retrieve(dataset, request, zip_name)
  
  print("-- Unzipping dataset...")
  with ZipFile(zip_name, 'r') as zip_ref:
    if len(zip_ref.filelist) > 1:
        print("Found multiple files:", *zip_ref.filelist, sep="\n")
    elif re.search(dt_vers.replace("_", "."), zip_ref.filelist[0].filename):
        print("File version check successful:", zip_ref.filelist[0].filename, sep="\n")
        zip_ref.extract(zip_ref.filelist[0], os.path.dirname(zip_name))
        srcf = os.path.join(root_folder, LandCV_Dir, in_Name, zip_ref.filelist[0].filename)
        os.rename(srcf, output_file)
else:
   print("NC File exists!")

2025-09-08 14:57:39,180 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


-- Unzipping dataset...
File version check successful:
C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1.area-subset.-32.892482205844416.-71.33041368117995.-33.291039620278845.-71.63790600801836.nc
